In [1]:
from stacksyms import parseDWARF, staticGDB, generateFeatures, generatePositionalEncodings

debugFilepath = 'data/cross-compile-dataset/bin/static/gcc/og/parallel'


In [8]:
module, importer = parseDWARF(debugFilepath)
firstUnit = module.components[0]
dwarfDB = importer._dwarf_db
dwarfData = dwarfDB._pri
print(dir(dwarfData))
print(dwarfData._die_map)
# TODO: we could get basic block info from DWARF.. maybe
#       look at "objdump --dwarf=line execFile" and DWARF Spec Section 6.2
#       
#lineProg = dwarfData.get_line_program()
#dir(lineProg)
#types = firstUnit.types
#globals = firstUnit.variables
functions = firstUnit.functions
print([func.name for func in functions])
#scopeQueries = ['info scope ' + func for func in functions]
disasQueries = ['disas /r ' + func.name for func in functions]
gdbOut = staticGDB(debugFilepath, functions, disasQueries)
#scopeResults, disasResults = results[0:len(results)//2], results[len(results)//2:]

01/23/2021 23:56:32 INFO:staticGDB:Loading file data/cross-compile-dataset/bin/static/gcc/og/parallel statically in GDB.


['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_arch', '_debug_root', '_debug_str', '_die_map', '_dwarf_info', '_elf_file', '_index', '_line_programs', '_location_lists', '_logger', '_range_lists', 'get_alt_filename', 'get_debug_str_at_offset', 'get_die_at_offset', 'get_line_program', 'get_location_list', 'get_range_list_at_offset', 'iter_compile_units', 'iter_section_starts']
{11: DIE DW_TAG_compile_unit, size=34, has_children=True
    |DW_AT_producer    :  AttributeValue(name='DW_AT_producer', form='DW_FORM_strp', value=b'GNU C11 6.3.0 20170406 -mtune=generic -march=x86-64 -g -Og -fstack-protector-strong', raw_value=2963, offset=12)
    |DW_AT_language    :  AttributeValue(name='DW_AT_langu

In [3]:
# TODO: newCollectLocals(gdbOut, scopeQueries, functions) and newCollectDisas(gdbOut, disasQueries, functions)
#for scope, func in zip(scopeResults, functions):
#    collectLocals(scope, functions[func])
for disas, func in zip(gdbOut, functions):
    func.disas = [tuple(line.strip().split('\\t')) for line in disas[1:-1]]

In [ ]:
X, Y, Z = [], [], []
for func in functions:
    print('////////////////////////')
    print(func.name, hex(func.start), func.frame_base)
    for lvar in func.variables:
        print(lvar.name, lvar.type, "(bytesize = %d)"%lvar.type.byte_size)
        for loc in lvar.locations:
            print(hex(loc.begin) + " to " + hex(loc.end) + ": " \
            + str(loc.type)[13:]
            + str(loc.expr))
        print('')
#    X += [func] #generateFeature(func, functions)]
#    Y += [generateLabel(func, functions)]
#    Z += [generateDebugLabel(func, functions)]
#print([x+" => "+ str(y) for x,y in zip(X,Z)])

#func = 'quotearg_n_style_colon'

In [9]:
main = list(filter(lambda x : x.name=='main', functions))[0]
main_x = generateFeatures(main)
main_penc = generatePositionalEncodings(main)
#print('START ' + ' '.join(main_x) + ' END')
print('\n'.join(map(lambda x : x[0] + ' ' + x[1], zip(main_penc, main_x))))

0x0000000000400f88 4157
0x0000000000400f8a 4156
0x0000000000400f8c 4155
0x0000000000400f8e 4154
0x0000000000400f90 55
0x0000000000400f91 53
0x0000000000400f92 4883ec48
0x0000000000400f96 4189ff
0x0000000000400f99 4989f4
0x0000000000400f9c 64488b042528000000
0x0000000000400fa5 4889442438
0x0000000000400faa 31c0
0x0000000000400fac 4863f7
0x0000000000400faf bf08000000
0x0000000000400fb4 e857460200
0x0000000000400fb9 4889442410
0x0000000000400fbe c744242001000000
0x0000000000400fc6 c744242402000000
0x0000000000400fce c744241c00000000
0x0000000000400fd6 c744241801000000
0x0000000000400fde f20f101d4a590a00
0x0000000000400fe6 f20f111c24
0x0000000000400feb c7442408ffffffff
0x0000000000400ff3 486305e6002d00
0x0000000000400ffa 498b14c4
0x0000000000400ffe 4885d2
0x0000000000401001 0f8422020000
0x0000000000401007 0fb60a
0x000000000040100a b82d000000
0x000000000040100f 29c8
0x0000000000401011 7513
0x0000000000401013 0fb64a01
0x0000000000401017 b82d000000
0x000000000040101c 29c8
0x000000000040101e 7

In [13]:
for func in functions:
    print('////////////////////////')
    print(func.name, func.frame_base)
    for lvar in func.variables:
        print(lvar.name, lvar.type, "(bytesize = %d)"%lvar.type.byte_size)
        for loc in lvar.locations:
    #        print(loc)
            print(hex(loc.begin) + " to " + hex(loc.end) + ": " \
    #            + str(loc.type)[13:]
                + str(loc.expr))
            print('')

////////////////////////
main Location(begin=0, end=0, type=<LocationType.STATIC_LOCAL: 1>, expr=(<ExprOp.CFA: 2>,))
maxjobs int (bytesize = 4)
0x400ff3 to 0x401099: (<ExprOp.CFA: 2>, -120, <ExprOp.ADD: 1>)

0x401099 to 0x4010b2: ('rax',)

0x4010fc to 0x40123e: (<ExprOp.CFA: 2>, -120, <ExprOp.ADD: 1>)

0x401254 to 0x401289: (<ExprOp.CFA: 2>, -120, <ExprOp.ADD: 1>)

0x401289 to 0x40128b: ('rax',)

0x40128b to 0x40154e: (<ExprOp.CFA: 2>, -120, <ExprOp.ADD: 1>)

curjobs int (bytesize = 4)
0x4013d8 to 0x401513: ('rbx',)

0x40151d to 0x40154e: ('rbx',)

maxload double (bytesize = 8)
0x401120 to 0x401178: ('xmm0',)

0x401512 to 0x40151d: (<ExprOp.CFA: 2>, -128, <ExprOp.ADD: 1>)

argsatonce int (bytesize = 4)
0x400ff3 to 0x4011ab: (<ExprOp.CFA: 2>, -104, <ExprOp.ADD: 1>)

0x4011ab to 0x4011c8: ('rax',)

0x4011c8 to 0x4013d8: (<ExprOp.CFA: 2>, -104, <ExprOp.ADD: 1>)

0x4013d8 to 0x401409: (<ExprOp.CFA: 2>, -116, <ExprOp.ADD: 1>)

0x401409 to 0x40154e: (<ExprOp.CFA: 2>, -104, <ExprOp.ADD: 1>)



TypeError: %d format: a number is required, not NoneType

In [ ]:
generateLabel(func, functions)

In [ ]:
cu = ELFFile(open(debugFilepath, mode='rb')).get_dwarf_info().iter_CUs().__next__()